In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
import snowflake.connector

# Replace these with your Snowflake account credentials and connection details
account = 'helpathome.east-us-2.azure'  # Replace with your Snowflake account URL
warehouse = 'DEMO_WH'
database = 'DISC_PROD'
schema = 'list_schema'
username = 'dsingh'  # Replace with your Snowflake username
password = 'Hah@2023'  # Replace with your Snowflake password

# Create the SQL_Files and View folders
sql_files_dir = "SQL_Files"
view_dir = os.path.join(sql_files_dir, "View")

if not os.path.exists(sql_files_dir):
    os.mkdir(sql_files_dir)

if not os.path.exists(view_dir):
    os.mkdir(view_dir)

# Establish a connection to Snowflake
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

# Create a cursor to execute SQL queries
cursor = conn.cursor()

list_schema = ['1UP_BISON','1UP_HEALTH','ACTIVE_DIRECTORY','ADP','ALAYACARE','APTITIVE_DEV','ASR','AXXESS','CCSI','CLEARCARE','COSTALSYNCDATA','DATAFLEXSYNCDATA','DATA_CLEANSING','DEVERO','FILEPRO',
'GENERATIONSALLIANCE','HHAEXCHANGEEDISON','HHAEXCHANGEOPENSYSTEMS','HHAEXCHANGEOSHAH','HHAEXCHANGEPREFERRED','JAZZHR','MATRIXCARE','MATRIXCARE_FULLLOAD','PUBLIC',
'SANDATAIMPORT','TRUSTPOINTDATA','TRUSTPOINTDATA_HIST','WORKDAY','ZENDESK'] #add schemas

views = [] # keep this empty for now

# Execute the query to get the list of views
for schema in list_schema:
    
    # Query Snowflake to get a list of views in the specified database and schema
    view_query = f'''
    SELECT (table_catalog || '.' || table_schema || '.' || table_name) table_name
    FROM information_schema.views
    WHERE table_schema = '{schema}' AND table_catalog = '{database}'
    '''
    cursor.execute(view_query)
    # Fetch the results
    view = cursor.fetchall()
    views.extend(view)


# Close the cursor and connection when done with the view query
cursor.close()

if views:
    # Iterate through the views and retrieve DDL statements
    for view_info in views:
        view_name = view_info[0]

        # Construct the fully qualified view name
        fully_qualified_view_name = f'{view_name}'

        # Query to retrieve the DDL statement for the view
        ddl_query = f'''
            SELECT GET_DDL('VIEW', '{view_name}', true)
        '''

        # Create a new cursor for the DDL query
        cursor = conn.cursor()

        # Execute the DDL query
        cursor.execute(ddl_query)

        # Fetch the DDL statement
        ddl_statement = cursor.fetchone()[0]


        # Modify the DDL statement to include the database and schema names
        modified_ddl_statement = ddl_statement.replace(f'create or replace view {view_name}',
                                                       f'create or replace view {fully_qualified_view_name}')

        # Remove line breaks and extra spaces
        modified_ddl_statement = ddl_statement.replace('\r\n', '\n')

        # Step 2: Create a .sql file and write the DDL statement to it
        sql_file_name = f"{view_dir}/{fully_qualified_view_name}.sql"
        with open(sql_file_name, 'w') as sql_file:
            sql_file.write(modified_ddl_statement)

        # Step 3: Print the file name (database.schema.view name)
        print(f"Generated SQL file: {fully_qualified_view_name}")

        # Close the cursor for the DDL query
        cursor.close()
else:
    print(f"No views found in the specified schema '{schema}' in database '{database}'.")

# Close the connection
conn.close()

Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_ALLERGYINTOLERANCE
Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_CONDITION
Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_COVERAGE
Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_DIAGNOSTICREPORT
Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_ENCOUNTER
Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_OBSERVATION
Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_ORGANIZATION
Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_PATIENT
Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_PRACTITIONER
Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_PROCEDURE
Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_SERVICEREQUEST
Generated SQL file: DISC_PROD.1UP_BISON.VW_FHIR_R4_RELATEDPERSON
Generated SQL file: DISC_PROD.1UP_HEALTH.VW_FHIR_R4_ALLERGYINTOLERANCE
Generated SQL file: DISC_PROD.1UP_HEALTH.VW_FHIR_R4_CONDITION
Generated SQL file: DISC_PROD.1UP_HEALTH.VW_FHIR_R4_COVERAGE
Generated SQL file: DISC_PROD.1UP_HEALTH.VW_